In [ ]:
!pip install transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [ ]:
from google.colab import files
uploaded = files.upload()  # original_shuffled.txt 업로드


Saving original_shuffled.txt to original_shuffled.txt


In [ ]:
from datasets import Dataset
import random

with open("original_shuffled.txt", "r", encoding="utf-8") as f:
    lines = [line.strip() for line in f if line.strip()]

random.shuffle(lines)
split1 = int(0.9 * len(lines))
split2 = int(0.95 * len(lines))
train_lines = lines[:split1]
val_lines = lines[split1:split2]
test_lines = lines[split2:]

train_dataset = Dataset.from_dict({"text": train_lines})
val_dataset = Dataset.from_dict({"text": val_lines})
test_dataset = Dataset.from_dict({"text": test_lines})


In [ ]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2", use_auth_token=False)
model = GPT2LMHeadModel.from_pretrained("gpt2", use_auth_token=False)

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1932: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:4112: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
def tokenize_function(example):
    encoding = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )
    encoding["labels"] = encoding["input_ids"]
    return encoding

tokenized_train = train_dataset.map(tokenize_function, remove_columns=["text"])
tokenized_val = val_dataset.map(tokenize_function, remove_columns=["text"])


Map:   0%|          | 0/156286 [00:00<?, ? examples/s]

Map:   0%|          | 0/8683 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results-original",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    save_total_limit=2,
    logging_steps=100,
    weight_decay=0.01,
    save_strategy="epoch",
    fp16=True,
    report_to="none",  # <<< ✅ 이 줄 추가하면 wandb 연결 안 함!
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,

)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train()


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,0.294400,0.295717
2,0.282800,0.282643
3,0.283500,0.278100


TrainOutput(global_step=58608, training_loss=0.30272888777655005, metrics={'train_runtime': 9384.9794, 'train_samples_per_second': 49.958, 'train_steps_per_second': 6.245, 'total_flos': 3.0627214884864e+16, 'train_loss': 0.30272888777655005, 'epoch': 3.0})

In [ ]:
trainer.save_model("checkpoint-original")


In [ ]:
trainer.save_model("checkpoint-original-final")


In [ ]:
import math
eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")


>>> Perplexity: 1.32


In [ ]:
!zip -r checkpoint-original-final.zip checkpoint-original-final
from google.colab import files
files.download("checkpoint-original-final.zip")


  adding: checkpoint-original-final/ (stored 0%)
  adding: checkpoint-original-final/config.json (deflated 52%)
  adding: checkpoint-original-final/training_args.bin (deflated 51%)
  adding: checkpoint-original-final/generation_config.json (deflated 24%)
  adding: checkpoint-original-final/model.safetensors (deflated 7%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>